At a very high level, MCMC tries to sample from an unknown target distribution $p(x)$ using a known function $f(x)$ where 
$$ p(x) = \frac{f(x)}{Z}$$
This is done in a Markov chain. One way is using the Metropolis Hastings algorithm. Essentially the algorithm samples the samples $x$ from a distribution $g(x)$ eg $g(x) = \mathcal{N}(x|\mu,\sigma)$, then samples the next sample depending on the previous sample $g(x_{t+1}|x_t)$. The sample generated in this step is accepted with probability of acceptance $A(x_t \rightarrow x_{t+1})$. 

If we can show that the probablity of going from any one state $x_t$ to another state $x_{t+1}$ follows the detail balance condition then we know that we have reached a stationary condition, and the Markov chain will be sampling from $p(x)$ for the next steps. The detail balance equation is simple $p(a) T(a \rightarrow b)=p(b) T(b \rightarrow q)$ where $T(a\rightarrow b)$ is the transition probablity for going from state a to state b. Since we don't know $p(a)$, we know $f(a)\approx p(a)$ with some proposed distribution $g(a|b)$ (eg a gaussian) to go from state $a$ to state $b$. 

$$\frac{f(a)}{Z} g(b / a) A(a \rightarrow b)=\frac{f(b)}{Z} g(a \mid b) A(b \rightarrow a)$$

From above we have 
$$\frac{A(a \rightarrow b)}{A(b \rightarrow a)}=\frac{f(b)}{f(a)} \times \frac{g(a \mid b)}{g(b \mid a)}$$

If $\frac{f(b)}{f(a)} \times \frac{g(a \mid b)}{g(b \mid a)} \lt 1$:

$A(a \rightarrow b)=\frac{f(b)}{f(a)} \times \frac{g(a \mid b)}{g(b \mid a)}$
$A(b \rightarrow a)=1$

If If $\frac{f(b)}{f(a)} \times \frac{g(a \mid b)}{g(b \mid a)} \ge 1$:


$A(a \rightarrow b)=1$
$A(b \rightarrow a)=\frac{f(a)}{f(b)} \times \frac{g(b \mid a)}{g(a \mid b)}$

If $g$ is symmetric (like a gaussian), then
$A(a \rightarrow b)=MIN \left(1, \frac{f(b)}{f(a)}\right.$
$=MIN \left(1, \frac{\rho(b)}{p(a)}\right)$


Questions: Can we use something like accept-reject algorithm, or minimization or other methods?

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
from scipy.stats import multivariate_normal
import seaborn as sns
import pymc3 as pm
import arviz as az
import subprocess as sb 
import emcee


plt.style.use('bmh')
colors = ['#348ABD', '#A60628', '#7A68A6', '#467821', '#D55E00', 
          '#CC79A7', '#56B4E9', '#009E73', '#F0E442', '#0072B2']
mp.rc('text', usetex=True)
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)
az.style.use("arviz-darkgrid")

%matplotlib inline

In [16]:
#LOAD DATA
chi2_array_ALL_DATA_25k = np.load('/home/ali/Desktop/Pulled_Github_Repositories/NNPDF_Uncertainty/master_version/local/ALL_DATA_25k/chi2_array_ALL_DATA_25k.npy')
MVN_25k_MASTER = np.load('/home/ali/Desktop/Pulled_Github_Repositories/NNPDF_Uncertainty/master_version/samples/MVN_25k_MASTER.npy')
COV_MASTER= np.load('/home/ali/Desktop/Pulled_Github_Repositories/NNPDF_Uncertainty/master_version/samples/COV_MASTER.npy')
params_MASTER= np.load('/home/ali/Desktop/Pulled_Github_Repositories/NNPDF_Uncertainty/master_version/samples/params_MASTER.npy')
params_MASTER.shape

(14,)

In [20]:
k=params_MASTER.reshape((1,14))
k[1]

IndexError: index 1 is out of bounds for axis 0 with size 1

in our case, $f(x) = e^{-0.5\chi^2}$, $p(x) = L(\theta)$ the true likelihood of the parameters. Toy example

In [8]:
COV_MASTER

array([[ 4.45e-06,  3.44e-06,  2.25e-06,  2.27e-06, -5.33e-06,  1.28e-06,
         1.23e-06,  3.32e-05, -2.41e-06, -2.81e-06,  5.41e-06, -9.82e-07,
        -2.68e-05,  2.50e-05],
       [ 3.44e-06,  2.09e-04,  2.02e-06, -8.84e-06, -2.75e-04,  5.14e-05,
        -3.97e-06, -1.59e-03, -9.26e-05, -7.11e-04, -1.85e-04, -1.42e-05,
         4.77e-04, -1.93e-05],
       [ 2.25e-06,  2.02e-06,  2.33e-06,  1.28e-06, -3.87e-06,  1.07e-06,
         3.89e-07,  1.25e-05, -3.32e-06, -5.05e-06,  1.64e-05, -2.39e-07,
        -3.22e-05,  5.76e-06],
       [ 2.27e-06, -8.84e-06,  1.28e-06,  4.30e-04,  1.51e-05, -2.65e-06,
        -9.05e-06,  1.76e-03,  1.25e-03,  9.56e-05,  4.52e-04,  1.49e-06,
        -3.11e-04, -3.76e-04],
       [-5.33e-06, -2.75e-04, -3.87e-06,  1.51e-05,  3.76e-04, -5.76e-05,
         6.79e-06,  2.24e-03,  1.37e-04,  1.02e-03,  2.55e-04,  2.10e-05,
        -6.84e-04,  2.35e-05],
       [ 1.28e-06,  5.14e-05,  1.07e-06, -2.65e-06, -5.76e-05,  3.33e-05,
        -1.47e-06, -3.01e-04, -

In [7]:
init_params = params_MASTER
sb.run("source /home/ali/Desktop/Research/xfitter/xfitter_master_version/setup.sh", shell =True)


/bin/sh: 1: source: not found


CompletedProcess(args='source /home/ali/Desktop/Research/xfitter/xfitter_master_version/setup.sh', returncode=127)

In [ ]:
init_params = params_MASTER
sb.run("source /home/ali/Desktop/Research/xfitter/xfitter_master_version/setup.sh", shell =True)

def ll(params):

    minuit_in_path = os.path.join(path, 'parameters.yaml')
    with open(minuit_in_path, 'w') as second:
        second.write('Minimizer: MINUIT # CERES \n')
        second.write('MINUIT:\n')
        second.write('  Commands: | \n')
        second.write('    call fcn 1\n')
        second.write('    set str 2\n')
        second.write('    call fcn 3\n')
        second.write('\n')
        second.write('Parameters:\n')
        second.write('  Ag   :  DEPENDENT\n')
        second.write('  Adbar   : [ ' + str(float(params[0])) + ', 0. ]\n')
        second.write('  Agp   : [ ' + str(format(float(params[1]), '.6f')) + ', 0. ]\n')
        second.write('  Bdbar   : [ ' + str(format(float(params[2]), '.6f')) + ', 0. ]\n')
        second.write('  Bdv   : [ ' + str(format(float(params[3]), '.6f')) + ', 0. ]\n')
        second.write('  Cgp   : [ ' + str(25.000) + ', 0. ]\n')
        #note that Cprig is a constant, not a parameter value!
        second.write('  Auv  :  DEPENDENT\n')
        second.write('  Bg   : [ ' + str(format(float(params[4]), '.6f')) + ', 0. ]\n')
        second.write('  Bgp   : [ ' + str(format(float(params[5]), '.6f')) + ', 0. ]\n')
        second.write('  Duv  : [    0     ]\n')
        second.write('  Buv   : [ ' + str(format(float(params[6]), '.6f')) + ', 0. ]\n')
        second.write('  Adv  :  DEPENDENT\n')
        second.write('  Cdbar   : [ ' + str(format(float(params[7]), '.6f')) + ', 0. ]\n')
        second.write('  Cdv   : [ ' + str(format(float(params[8]), '.6f')) + ', 0. ]\n')
        second.write('  Aubar: [ 0.0, 0.0 ]\n')
        second.write('  Bubar: [ 0.0, 0.0  ]\n')
        second.write('  Cg   : [ ' + str(format(float(params[9]), '.6f')) + ', 0. ]\n')
        second.write('  Cubar   : [ ' + str(format(float(params[10]), '.6f')) + ', 0. ]\n')
        second.write('  Cuv   : [ ' + str(format(float(params[11]), '.6f')) + ', 0. ]\n')
        second.write('  Dubar   : [ ' + str(format(float(params[12]), '.6f')) + ', 0. ]\n')
        second.write('  Euv   : [ ' + str(format(float(params[13]), '.6f')) + ', 0. ]\n')
        second.write('\n')

        second.write('  ZERO : [ 0. ]\n')        
        second.write('  fs   :   [ 0.4, 0.0 ]\n')
        #second.write('  DbarToS: \"=fs/(1-fs)\"\n')

        second.write('Parameterisations:\n')
        second.write('  par_uv:\n')
        second.write('    class: HERAPDF\n')
        second.write('    parameters: [Auv,Buv,Cuv,Duv,Euv]\n')
        second.write('  par_dv:\n')
        second.write('    class: HERAPDF\n')
        second.write('    parameters: [Adv,Bdv,Cdv]\n')
        second.write('  par_ubar:\n')
        second.write('    class: HERAPDF\n')
        second.write('    parameters: [Adbar,Bdbar,Cubar,Dubar]\n')
        second.write('  par_dbar:\n')
        second.write('    class: HERAPDF\n')        
        second.write('    parameters: [Adbar,Bdbar,Cdbar]\n')
        second.write('  par_s:\n')
        second.write('    class: Expression\n')
        second.write('    expression: \"Adbar*fs/(1-fs)*(x^Bdbar*(1-x)^Cdbar)\"\n')
        # second.write('    input: par_dbar\n')
        second.write('\n')
        second.write('  par_g:\n')
        second.write('    class: NegativeGluon\n')
        second.write('    parameters: [Ag,Bg,Cg,ZERO,ZERO,Agp,Bgp,Cgp]\n')
        second.write('\n\n')
        second.write('DefaultDecomposition: proton\n')
        second.write('Decompositions:\n')
        second.write('  proton:\n')
        second.write('    class: UvDvUbarDbarS\n')
        second.write('    xuv: par_uv\n')
        second.write('    xdv: par_dv\n')
        second.write('    xubar: par_ubar\n')
        second.write('    xdbar: par_dbar\n')
        second.write('    xs: par_s\n')
        second.write('    xg: par_g\n')
        second.write('\n')
        second.write('DefaultEvolution: proton-QCDNUM\n')
        second.write('\n\n')
        second.write('Evolutions:\n')
        # second.write('  proton-APFELff:\n')
        # second.write('    ? !include evolutions/APFEL.yaml\n')
        # second.write('    decomposition: proton\n')

        second.write('  proton-QCDNUM:\n')
        second.write('    ? !include evolutions/QCDNUM.yaml\n')
        second.write('    decomposition: proton\n')

        # second.write('  antiproton:\n')
        # second.write('    class: FlipCharge\n')
        # second.write('  neutron:\n')
        # second.write('    class: FlipUD\n')   

        # second.write('  proton-LHAPDF:\n')
        # second.write('    class: LHAPDF\n')
        # second.write('    set: \"NNPDF30_nlo_as_0118\"\n')
        # second.write('    member: 0\n')
        second.write('\n')

        second.write('Q0 : 1.378404875209\n')
        second.write('\n')
        second.write('? !include constants.yaml\n')
        second.write('\n')
        second.write('alphas : 0.118\n')
        second.write('\n')
        second.write('byReaction:\n')
        second.write('\n')        
        second.write('  RT_DISNC:\n')
        second.write('    ? !include reactions/RT_DISNC.yaml\n')
        second.write('  FONLL_DISNC:\n')
        second.write('    ? !include reactions/FONLL_DISNC.yaml\n')
        second.write('  FONLL_DISCC:\n')
        second.write('    ? !include reactions/FONLL_DISCC.yaml\n')
        second.write('  FFABM_DISNC:\n')
        second.write('    ? !include reactions/FFABM_DISNC.yaml\n')
        second.write('  FFABM_DISCC:\n')
        second.write('    ? !include reactions/FFABM_DISCC.yaml\n')
        # second.write('  AFB:\n')
        # second.write('    ? !include reactions/AFB.yaml \n')
        second.write('  APPLgrid:\n')
        second.write('    ? !include reactions/APPLgrid.yaml\n')
        second.write('  Fractal_DISNC:\n')
        second.write('    ? !include reactions/Fractal_DISNC.yaml\n')
        second.write('\n\n')
        second.write('hf_scheme_DISNC :\n')
        second.write('  defaultValue : \'RT_DISNC\' \n')
        second.write('\n')
        second.write('hf_scheme_DISCC :\n')
        second.write('  defaultValue : \'BaseDISCC\' \n')
        second.write('\n')


    #this should still work because we are not minimizing the chi2, we are just inputting the fixed parameter values in the minuit.in.txt format
    #RUN XFITTER AND PIPE ITS OUTPUT TO S
    s = os.popen("xfitter").read()
    #this executes the command xfitter, and the command that will go to the screen will be captured here

    #THIS IS THE EXPRESSION FORM: 
    # @chi2out__   503.08321706305105     

    #pattern = re.compile('[@chi2out].[0-9]+[.][0-9]+')
    pattern = re.compile('[@chi2out].[\d+]+[.][\d+]+'); regex=r'@chi2out__...\d+\.\d+'
    #matches = pattern.finditer(s)
    matches = re.findall(regex, s, re.MULTILINE)
    #print(matches)

    chi2 = matches[0].split()[1]

    return np.exp(-0.5 * float(chi2))  

ndim=14
nwalkers = 4
p0 = np.random.rand(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, log, args=init_params)

state = sampler.run_mcmc(p0, 100)
sampler.reset()
#100 is the burn in 
sampler.run_mcmc(state, 10000)
#10000 steps

samples = sampler.get_chain(flat=True)
print(samples)

In [14]:
added = np.ones((1,14))
m = np.vstack((MVN_25k_MASTER, added))
m[-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])